In [1]:
import numpy as np
import sys
sys.path.append('../')
from utils.sparse_vector import *
from utils.vector_polynomial_isomorphism import *
from utils.minor_flattening import *

$\hat{\sigma_{2}}(\mathbb{P}^{1} \times \mathbb{P}^{1} \times \mathbb{P}^{1} \times \mathbb{P}^{2})$

In [2]:
# Let us initialize constants and vector spaces.
N = 2*2*2*3
n = [2,2,2,3]
d = 3
indeterminates = form_indeterminates(n)
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, d)

Defining x_0000, x_0001, x_0002, x_0010, x_0011, x_0012, x_0100, x_0101, x_0102, x_0110, x_0111, x_0112, x_1000, x_1001, x_1002, x_1010, x_1011, x_1012, x_1100, x_1101, x_1102, x_1110, x_1111, x_1112


In [3]:
f = get_flattening_minors(R, n, d)
f_vec = [vp_iso.polynomial_to_vector(fi) for fi in f]

In [4]:
W = V.subspace(f_vec)

print(W)

Vector space of degree 2600 and dimension 184 over Rational Field
Basis matrix:
184 x 2600 dense matrix over Rational Field


In [6]:
W_perp = W.basis_matrix().right_kernel_matrix(algorithm='default', basis='computed')
print(W_perp)

p = W.dimension()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
print(W_perp.ncols())

def E_action(a, b, grad_fl):
    sum = -1*indeterminates[b]*grad_fl[a]
    return R(sum)

2600


We note `M_f` $:= \tilde{M}_f$

In [7]:
g_sparse = [sparsify(W_perp.gen(k)) for k in range(q)]

In [23]:
import threading
import multiprocessing

# Construct M_f
M_f = zero_matrix(QQ, W.dimension()*q, N^2, sparse=True)

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

process_ranges = list(split(range(W.dimension()), 8))

def parallelize(given_range, processnum):
    E_action_fl = [[0 for j in range(N)] for i in range(N)]
    for l in given_range:
        print(processnum, l)
        grad_fl = vp_iso.vector_to_polynomial(W.gen(l)).gradient()
        for i in range(N):
            for j in range(N):
                E_action_fl[i][j] = sparsify(vp_iso.polynomial_to_vector(E_action(i,j,grad_fl)))
        for k in range(q):
            for i in range(N):
                for j in range(N):
                    M_f[(q*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse[k])
    
threads = []
    
for i in range(8):
    p = threading.Thread(target=parallelize, args=[process_ranges[i], i])
    p.start()
    threads.append(p)
    
for t in threads:
    t.join()

0 01
 23
2 46
3 69
4 92
5 115
6 138
7 161
4 93
6 139
7 162
5 116
1 24
2 47
0 13 70

4 94
6 140
7 163
5 117
4 95
7 164
1 25
6 141
2 48
5 118
3 71
0 2
4 96
6 142
7 165
5 119
2 49
3 72
1 26
0 3
4 97
6 143
7 166
5 120
7 167
4 98
5 121
6 144
2 50
1 27
3 73
0 4
7 168
4 99
5 122
6 145
3 1 74
28
2 51
7 169
4 100
0 5
5 123
6 146
7 170
4 101
6 147
5 124
3 75
2 52
1 29
0 6
4 102
7 171
6 148
5 125
4 103
7 172
0 7
2 53
6 149
3 76
5 126
1 30
4 104
7 173
6 150
5 127
2 54
0 8
3 77
1 31
7 174
4 105
6 151
5 128
7 175
4 106
6 152
5 129
0 9
2 55
3 78
1 32
7 176
4 107
6 153
5 130
7 177
0 10
2 56
1 33
64 108
 154
3 79
5 131
7 178
6 155
4 109
5 132
0 11
2 57
3 80
1 34
7 179
6 156
4 110
5 133
7 180
2 58
6 0 15712

3 81
1 35
4 111
5 134
7 181
6 158
4 112
5 135
2 59
0 13
1 36
3 82
7 182
6 159
4 113
5 136
7 183
2 60
6 160
0 14
3 83
1 37
4 114
5 137
1 38
3 84
0 15
2 61
3 85
2 62
0 16
1 39
3 86
0 17
2 63
1 40
0 18
3 87
1 41
2 64
0 19
3 88
1 42
2 65
0 20
3 89
1 43
2 66
0 21
1 44
3 90
2 67
0 22
1 45
3 91
2 68


In [24]:
print(M_f.is_zero())
rank = M_f.rank()

print('rank(M_f) =', rank)
print('dim(ker(M_f)) =', glN.dimension() - rank)

False
rank(M_f) = 558
dim(ker(M_f)) = 18


In [19]:
print('expected:', sum([ni^2-1 for ni in n]) + 1)

expected: 18
